In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

# Function to extract text or attribute values from BeautifulSoup objects
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

In [2]:
# Lists to store data
users = []
# userReviewNum = []
ratings = []
locations = []
dates = []
reviews = []
ReviewTitles = []

# Page range and company name
from_page = 1
to_page = 94
company = 'heymondo.com'

In [3]:
# # Loop through pages and extract data
# for i in range(from_page, to_page + 1):
#     result = requests.get(fr"https://www.trustpilot.com/review/{company}?page={i}")
#     soup = BeautifulSoup(result.content, 'html.parser')

#     # Extract data from each section of the page
#     soup2list(soup.find_all('span', {'class':'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
#     soup2list(soup.find_all('div', {'class':'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
#     soup2list(soup.find_all('time', {'data-service-review-date-time-ago':'true'}), dates, attr='datetime')
#     soup2list(soup.find_all('div', {'class':'styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
#     soup2list(soup.find_all('div', {'class':'styles_reviewContent__0Q2Tg'}), reviews)
#     soup2list(soup.find_all('h2', {'class':'typography_heading-s__f7029 typography_appearance-default__AAY17'}), ReviewTitles)
#     # Sleep to avoid being throttled
#     sleep(1)
    

# Loop through pages and extract data
for i in range(from_page, to_page + 1):
    result = requests.get(fr"https://www.trustpilot.com/review/{company}?page={i}")
    soup = BeautifulSoup(result.content, 'html.parser')

    # Extract data from each section of the page
    soup2list(soup.find_all('span', {'class':'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
    soup2list(soup.find_all('div', {'class':'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
    soup2list(soup.find_all('time', {'data-service-review-date-time-ago':'true'}), dates, attr='datetime')
    soup2list(soup.find_all('div', {'class':'styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
    
    # Get review content while ignoring the 'Date of experience' section
    reviews_and_dates = soup.find_all('div', {'class': 'styles_reviewContent__0Q2Tg'})
    for review_div in reviews_and_dates:
        # Append review text
        review_text = review_div.find('p', {'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'})
        if review_text:
            reviews.append(review_text.get_text(strip=True))

    # Extract Review Titles
    soup2list(soup.find_all('h2', {'class':'typography_heading-s__f7029 typography_appearance-default__AAY17'}), ReviewTitles)
    
    # Sleep to avoid being throttled
    sleep(1)


In [4]:
# Handle different lengths of lists by padding them with None
max_length = max(len(users), len(locations), len(dates), len(reviews), len(ratings),len(ReviewTitles))

In [5]:
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to the maximum length
users = pad_list(users, max_length)
locations = pad_list(locations, max_length)
dates = pad_list(dates, max_length)
reviews = pad_list(reviews, max_length)
ratings = pad_list(ratings, max_length)
ReviewTitles = pad_list(ReviewTitles, max_length)

# Create the DataFrame
review_data = pd.DataFrame(
{
    'Username': users,
    'Location': locations,
    'Date': dates,
    'Reviews': reviews,
    'Rating': ratings,
    'Review Titles': ReviewTitles
})

In [6]:
# Save the DataFrame to a CSV file
review_data.to_csv('reviews.csv', index=False)

In [7]:
file_path = r'./reviews.csv'
df = pd.read_csv(file_path)

In [8]:
df.head()

,Username,Location,Date,Reviews,Rating,Review Titles
0,Emma,GB,2024-10-01T10:14:48.000Z,Effective and communicative . Responded to all...,5,Effective and communicative
1,Doina Serban,GB,2024-09-25T22:46:14.000Z,"First of all, it is easy to do the ensurance o...",5,"Good quality, easy access"
2,Celia,CA,2024-09-18T01:22:00.000Z,It's so easy to purchase this insurance online...,5,A repeat customer!
3,customer,SE,2024-10-02T06:56:48.000Z,So far so good and I hope I don't need to find...,5,So far so good and I hope I don't need…
4,Yvonne Buhrmann,ID,2024-09-13T01:51:51.000Z,We have insurance at Heymondo and want to upgr...,2,No one can actually help you or knows whats go...
